In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

# Sitemap
Extends from the WebBaseLoader, SitemapLoader loads a sitemap from a given URL, and then scrapes and loads all pages in the sitemap, returning each page as a Document.

The scraping is done concurrently. There are reasonable limits to concurrent requests, defaulting to 2 per second. If you aren't concerned about being a good citizen, or you control the scrapped server, or don't care about load you can increase this limit. Note, while this will speed up the scraping process, it may cause the server to block you. Be careful!

### Fix notebook asyncio bug

In [2]:
import nest_asyncio

nest_asyncio.apply()

# Initialization
Now we can instantiate our model object and load documents:

In [4]:
from langchain_community.document_loaders.sitemap import SitemapLoader

In [5]:
sitemap_loader = SitemapLoader(web_path="https://api.python.langchain.com/sitemap.xml")

# Load

In [6]:
docs = sitemap_loader.load()
docs[0]

Fetching pages: 100%|##########| 18/18 [00:08<00:00,  2.22it/s]


Document(metadata={'source': 'https://api.python.langchain.com/en/latest/', 'loc': 'https://api.python.langchain.com/en/latest/', 'lastmod': '2024-10-25T14:03:14.666397+00:00', 'changefreq': 'weekly', 'priority': '1'}, page_content='\n\n\n\n\n\n\n\n\n\nLangChain Python API Reference Documentation.\n\n\nYou will be automatically redirected to the new location of this page.\n\n')

In [7]:
print(docs[0].metadata)

{'source': 'https://api.python.langchain.com/en/latest/', 'loc': 'https://api.python.langchain.com/en/latest/', 'lastmod': '2024-10-25T14:03:14.666397+00:00', 'changefreq': 'weekly', 'priority': '1'}


You can change the ```requests_per_second``` parameter to increase the max concurrent requests. and use ```requests_kwargs``` to pass kwargs when send requests.

In [8]:
sitemap_loader.requests_per_second = 2
# Optional: avoid `[SSL: CERTIFICATE_VERIFY_FAILED]` issue
sitemap_loader.requests_kwargs = {"verify": False}

# Lazy Load
You can also load the pages lazily in order to minimize the memory load.

In [9]:
page = []
for doc in sitemap_loader.lazy_load():
    page.append(doc)
    if len(page) >= 10:
        # do some paged operation, e.g.
        # index.upsert(page)

        page = []

c:\Code\Github\LangChain-world\venv\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.python.langchain.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Fetching pages: 100%|##########| 18/18 [00:00<00:00, 35.27it/s]


# Filtering sitemap URLs
Sitemaps can be massive files, with thousands of URLs. Often you don't need every single one of them. You can filter the URLs by passing a list of strings or regex patterns to the ```filter_urls``` parameter. Only URLs that match one of the patterns will be loaded.

In [11]:
loader = SitemapLoader(
    web_path="https://api.python.langchain.com/sitemap.xml",
    filter_urls=["https://api.python.langchain.com/en/latest"],
)


Fetching pages: 100%|##########| 1/1 [00:00<00:00, 19.23it/s]


In [ ]:
documents = loader.load()

In [12]:
documents[0]

Document(metadata={'source': 'https://api.python.langchain.com/en/latest/', 'loc': 'https://api.python.langchain.com/en/latest/', 'lastmod': '2024-10-25T14:03:14.666397+00:00', 'changefreq': 'weekly', 'priority': '1'}, page_content='\n\n\n\n\n\n\n\n\n\nLangChain Python API Reference Documentation.\n\n\nYou will be automatically redirected to the new location of this page.\n\n')

# Add custom scraping rules
The ```SitemapLoader``` uses ```beautifulsoup4``` for the scraping process, and it scrapes every element on the page by default. The ```SitemapLoader``` constructor accepts a custom scraping function. This feature can be helpful to tailor the scraping process to your specific needs; for example, you might want to avoid scraping headers or navigation elements.

The following example shows how to develop and use a custom function to avoid navigation and header elements.

Import the ```beautifulsoup4``` library and define the custom function.

In [13]:
from bs4 import BeautifulSoup


def remove_nav_and_header_elements(content: BeautifulSoup) -> str:
    # Find all 'nav' and 'header' elements in the BeautifulSoup object
    nav_elements = content.find_all("nav")
    header_elements = content.find_all("header")

    # Remove each 'nav' and 'header' element from the BeautifulSoup object
    for element in nav_elements + header_elements:
        element.decompose()

    return str(content.get_text())

Add your custom function to the ```SitemapLoader``` object.



In [14]:
loader = SitemapLoader(
    "https://api.python.langchain.com/sitemap.xml",
    filter_urls=["https://api.python.langchain.com/en/latest/"],
    parsing_function=remove_nav_and_header_elements,
)

In [16]:
documents = loader.load()

Fetching pages: 100%|##########| 1/1 [00:00<00:00, 18.99it/s]


In [17]:
documents[0]

Document(metadata={'source': 'https://api.python.langchain.com/en/latest/', 'loc': 'https://api.python.langchain.com/en/latest/', 'lastmod': '2024-10-25T14:03:14.666397+00:00', 'changefreq': 'weekly', 'priority': '1'}, page_content='\n\n\n\n\n\n\n\n\n\nLangChain Python API Reference Documentation.\n\n\nYou will be automatically redirected to the new location of this page.\n\n')

# Local Sitemap
The sitemap loader can also be used to load local files.



In [15]:
sitemap_loader = SitemapLoader(web_path="example_data/sitemap.xml", is_local=True)

docs = sitemap_loader.load()

FileNotFoundError: [Errno 2] No such file or directory: 'example_data/sitemap.xml'